<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Create-Model" data-toc-modified-id="Create-Model-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Create Model</a></span></li><li><span><a href="#Training" data-toc-modified-id="Training-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Training</a></span></li></ul></div>

In [1]:
import sys
import random
import os
sys.path.insert(0, '/home/abanihi/devel/clones/Mask_RCNN/')

In [2]:
from config import Config
import utils
import model as modellib
import visualize
from model import log

Using TensorFlow backend.


In [3]:
# Root directory of the project
ROOT_DIR = os.getcwd()

# Local path to trained weights file
COCO_MODEL_PATH = "/home/abanihi/devel/clones/Mask_RCNN/mask_rcnn_coco.h5"
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")


In [4]:
class NucleiConfig(Config):

    # Give the configuration a recognizable name
    NAME = "Nuclei"

    # Train on 1 GPU and 2 images per GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 16

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # background + 1

    IMAGE_MIN_DIM = 128
    IMAGE_MAX_DIM = 128

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  # anchor side in pixels

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 32

    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 10

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 5

In [5]:
config = NucleiConfig()
config.display()


Configurations:
BACKBONE_SHAPES                [[32 32]
 [16 16]
 [ 8  8]
 [ 4  4]
 [ 2  2]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     16
BBOX_STD_DEV                   [ 0.1  0.1  0.2  0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
IMAGES_PER_GPU                 16
IMAGE_MAX_DIM                  128
IMAGE_MIN_DIM                  128
IMAGE_PADDING                  True
IMAGE_SHAPE                    [128 128   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [ 123.7  116.8  103.9]
MINI_MASK_SHAPE                (56, 56)
NAME                           Nuclei
NUM_CLASSES                    2
POOL_SIZE                      7
POST_NMS_ROIS_INFERENCE        1000
POST_NMS_ROIS_TRAINING         

In [6]:
import numpy as np

In [7]:
images_train = np.load('../data/X_train.npy')
masks_train = np.load('../data/Y_train.npy')
# Get train and test IDs
train_ids = next(os.walk('../data/stage1_train'))[1]
len(train_ids)

670

In [8]:
images_train.shape, masks_train.shape

((670, 128, 128, 3), (670, 128, 128, 1))

In [9]:
from sklearn.utils import shuffle

In [10]:
images, masks, ids = shuffle(images_train, masks_train, train_ids, random_state=42)

In [11]:
x_train = images[:502]
y_train = masks[:502]
ids_train = ids[:502]
x_val = images[502:]
y_val = masks[502:]
ids_val = ids[502:]

In [12]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape

((502, 128, 128, 3),
 (502, 128, 128, 1),
 (168, 128, 128, 3),
 (168, 128, 128, 1))

In [14]:
class NucleiDataset(utils.Dataset):
    def load_nuclei(self, dataset_tensor, mask_tensor, class_ids):

        # Add class
        self.add_class("nuclei_mask", 1, "nuclei")

        # Add images
        for i in class_ids:
            self.add_image("nuclei_mask", image_id=i, path=None,
                           width=dataset_tensor.shape[1], height=dataset_tensor.shape[2])
            
    def load_image(self, image_id):
        info = self.image_info[image_id]
        return dataset_tensor[image_id]

    def load_mask(self, image_id):
        pass

    def image_reference(self, image_id):
        info = self.image_info[image_id]
        if info["source"] == "nuclei_mask":
            return info["nuclei_mask"]
        else:
            super(self.__class__).image_reference(self, image_id)

In [15]:
da = NucleiDataset()
da.load_nuclei(x_train, y_train, ids_train)

In [17]:
da.load_image()

TypeError: load_image() missing 1 required positional argument: 'image_id'

# Create Model

In [ ]:
model = modellib.MaskRCNN(mode="training", config=config, model_dir=MODEL_DIR)

In [ ]:
# Which weights to start with?
init_with = "coco"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last()[1], by_name=True)


# Training

Train in two stages:

- Only the heads. Here we're freezing all the backbone layers and training only the randomly initialized layers (i.e. the ones that we didn't use pre-trained weights from MS COCO). To train only the head layers, pass layers='heads' to the train() function.

- Fine-tune all layers. For this simple example it's not necessary, but we're including it to show the process. Simply pass layers="all to train all layers.


In [ ]:
model.train()